In [ ]:
#!/usr/bin/env python3
"""
Informatica PowerCenter XML to DBT Project Conversion Script - Production Version v2
Enhanced with explicit transformation-to-model mapping and improved prompting
Processes XML files from Snowflake stage and converts them to DBT projects using Claude-4-Sonnet
"""

import json
import re
import time
import logging
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
from collections import defaultdict
from dataclasses import dataclass
from functools import wraps
import traceback

from snowflake.snowpark.context import get_active_session
from snowflake.cortex import complete
from snowflake.snowpark.functions import col, lit

# ============================================================================
# CONFIGURATION MANAGEMENT
# ============================================================================

@dataclass
class Config:
    """Centralized configuration for the conversion process"""
    # Stage Configuration
    INPUT_STAGE: str = "@XML_INPUT"
    OUTPUT_STAGE: str = "@DBT_OUTPUT"
    
    # Model Configuration - UPDATED FOR CLAUDE 4 SONNET
    #CLAUDE_MODEL: str = "claude-sonnet-4-5"
    CLAUDE_MODEL: str = "claude-4-sonnet"    # Claude model for conversion
    MAX_TOKENS: int = 80000  # Safe limit for Claude 4 Sonnet (200k context)
    CHUNK_SIZE: int = 75000  # Character limit per chunk
    
    # Processing Configuration
    BATCH_SIZE: int = 5
    MAX_RETRIES: int = 3
    RETRY_DELAY: int = 10  # seconds
    RATE_LIMIT_CALLS_PER_MINUTE: int = 1
    
    # Database Configuration
    OUTPUT_TABLE: str = "TPC_DI_RAW_DATA.DBT_INGEST.INFORMATICA_DBT_CONVERTED_FILES_MAIN_DEMO"
    SUMMARY_TABLE: str = "TPC_DI_RAW_DATA.DBT_INGEST.INFORMATICA_DBT_CONVERSION_SUMMARY_MAIN_DEMO"
    FILE_FORMAT: str = "xml_format"
    
    # Logging Configuration
    LOG_LEVEL: str = "INFO"

# Initialize global config
config = Config()

# ============================================================================
# LOGGING SETUP
# ============================================================================

def setup_logging(level: str = "INFO") -> logging.Logger:
    """Setup structured logging"""
    logger = logging.getLogger("informatica_dbt_converter")
    logger.setLevel(getattr(logging, level))
    
    # Clear existing handlers
    logger.handlers.clear()
    
    # Console handler
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger

logger = setup_logging(config.LOG_LEVEL)

# ============================================================================
# CUSTOM EXCEPTIONS
# ============================================================================

class ConversionError(Exception):
    """Base exception for conversion errors"""
    pass

class XMLReadError(ConversionError):
    """Error reading XML content"""
    pass

class ClaudeAPIError(ConversionError):
    """Error calling Claude API"""
    pass

class ValidationError(ConversionError):
    """Error validating generated content"""
    pass

# ============================================================================
# DECORATORS AND UTILITIES
# ============================================================================

def rate_limit(calls_per_minute: int = 1):
    """Rate limiting decorator for API calls"""
    min_interval = 60.0 / calls_per_minute
    last_called = [0.0]
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            elapsed = time.time() - last_called[0]
            if elapsed < min_interval:
                sleep_time = min_interval - elapsed
                logger.debug(f"Rate limiting: sleeping for {sleep_time:.2f}s")
                time.sleep(sleep_time)
            result = func(*args, **kwargs)
            last_called[0] = time.time()
            return result
        return wrapper
    return decorator

def retry_on_failure(max_retries: int = 3, delay: int = 2):
    """Retry decorator with exponential backoff"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries - 1:
                        raise
                    wait_time = delay * (2 ** attempt)
                    logger.warning(
                        f"Attempt {attempt + 1} failed: {str(e)}. "
                        f"Retrying in {wait_time}s..."
                    )
                    time.sleep(wait_time)
            return None
        return wrapper
    return decorator

def track_time(operation_name: str):
    """Context manager for tracking operation time"""
    class TimeTracker:
        def __enter__(self):
            self.start = time.time()
            return self
        
        def __exit__(self, *args):
            duration = time.time() - self.start
            logger.info(f"{operation_name} completed in {duration:.2f}s")
    
    return TimeTracker()

# ============================================================================
# SNOWFLAKE SESSION MANAGEMENT
# ============================================================================

def get_snowflake_session():
    """Initialize Snowflake session with error handling"""
    try:
        session = get_active_session()
        logger.info(
            f"Connected to Snowflake - "
            f"Database: {session.get_current_database()}, "
            f"Schema: {session.get_current_schema()}"
        )
        return session
    except Exception as e:
        logger.error(f"Failed to connect to Snowflake: {e}")
        raise

# ============================================================================
# FILE DISCOVERY AND READING
# ============================================================================

def list_xml_files_with_folders(session, stage_name: str) -> Dict[str, List[str]]:
    """List XML files grouped by folder in the specified stage"""
    try:
        with track_time("Listing XML files"):
            files_result = session.sql(f"LIST {stage_name}").collect()
            
            folder_files = defaultdict(list)
            
            for row in files_result:
                file_path = row['name']
                if file_path.lower().endswith('.xml'):
                    parts = file_path.split('/')
                    folder_name = parts[1] if len(parts) > 2 else "root"
                    folder_files[folder_name].append(file_path)
            
            total_files = sum(len(files) for files in folder_files.values())
            logger.info(
                f"Found {total_files} XML files in {len(folder_files)} folders")
            
            for folder, files in folder_files.items():
                logger.debug(f"  Folder '{folder}': {len(files)} files")
            
            return dict(folder_files)
    
    except Exception as e:
        logger.error(f"Error listing files from {stage_name}: {e}")
        raise

def sanitize_file_path(file_path: str, stage_prefix: str = 'xml_input/') -> str:
    """Sanitize file path by removing stage prefix"""
    if file_path.startswith(stage_prefix):
        return file_path[len(stage_prefix):]
    return file_path

@retry_on_failure(max_retries=3, delay=2)
def read_xml_content(session, stage_name: str, file_path: str) -> Optional[str]:
    """Read XML file content from Snowflake stage with retries"""
    try:
        clean_path = sanitize_file_path(file_path)
        
        # Use parameterized approach for safer queries
        query = f"SELECT $1 FROM {stage_name}/{clean_path} (FILE_FORMAT => '{config.FILE_FORMAT}')"
        
        logger.debug(f"Reading file: {clean_path}")
        xml_rows = session.sql(query).collect()
        
        xml_content = "\n".join([row[0] for row in xml_rows if row[0]])
        
        if not xml_content.strip():
            logger.warning(f"Empty content for {clean_path}")
            raise XMLReadError(f"Empty content for {clean_path}")
        
        content_size = len(xml_content)
        estimated_tokens = estimate_token_count(xml_content)
        logger.info(
            f"Read {content_size:,} characters "
            f"(~{estimated_tokens:,} tokens) from {clean_path}"
        )
        
        return xml_content
        
    except Exception as e:
        logger.error(f"Error reading {file_path}: {e}")
        raise XMLReadError(f"Failed to read {file_path}: {str(e)}")

# ============================================================================
# TOKEN ESTIMATION AND CHUNKING
# ============================================================================

def estimate_token_count(text: str) -> int:
    """
    Improved token count estimation for Claude models
    Uses a more accurate heuristic based on character types
    """
    if not text:
        return 0
    
    # Count different character types for better estimation
    whitespace_count = sum(1 for c in text if c.isspace())
    alphanumeric_count = sum(1 for c in text if c.isalnum())
    special_count = len(text) - whitespace_count - alphanumeric_count
    
    # Weighted estimation (whitespace tokens are cheaper)
    estimated = (alphanumeric_count + special_count) / 3.5 + whitespace_count / 6
    
    # Add 25% buffer for XML structure overhead
    return int(estimated * 1.20)

def extract_xml_sections(xml_content: str) -> List[Tuple[str, str]]:
    """
    Extract logical sections from XML preserving hierarchical structure
    Key improvement: Keep MAPPING with all child elements intact
    """
    sections = []
    
    # Strategy: Extract top-level logical units that preserve relationships
    # Priority order ensures we get complete structures
    
    patterns = [
        # 1. Extract complete MAPPING blocks (contains transformations, instances, connectors)
        (r'<MAPPING\s+[^>]*>.*?</MAPPING>', 'MAPPING'),
        
        # 2. Extract complete WORKFLOW blocks (contains sessions, tasks)
        (r'<WORKFLOW\s+[^>]*>.*?</WORKFLOW>', 'WORKFLOW'),
        
        # 3. Extract SOURCE definitions (usually in shared folders)
        (r'<SOURCE\s+[^>]*>.*?</SOURCE>', 'SOURCE'),
        
        # 4. Extract TARGET definitions (usually in shared folders)
        (r'<TARGET\s+[^>]*>.*?</TARGET>', 'TARGET'),
        
        # 5. Extract SHORTCUT references (point to shared objects)
        (r'<SHORTCUT\s+[^>]*/>|<SHORTCUT\s+[^>]*>.*?</SHORTCUT>', 'SHORTCUT'),
    ]
    
    # Track extracted positions to avoid duplicates
    extracted_ranges = []
    
    for pattern, section_type in patterns:
        matches = re.finditer(pattern, xml_content, re.DOTALL)
        for match in matches:
            start, end = match.span()
            
            # Check if this range overlaps with already extracted sections
            is_overlap = False
            for existing_start, existing_end in extracted_ranges:
                if not (end <= existing_start or start >= existing_end):
                    is_overlap = True
                    break
            
            if not is_overlap:
                sections.append((match.group(0), section_type))
                extracted_ranges.append((start, end))
                logger.debug(f"Extracted {section_type} section: {start}-{end} ({end-start} chars)")
    
    if not sections:
        sections.append((xml_content, 'COMPLETE'))
        logger.warning("No specific sections found, using complete XML")
    
    logger.info(f"Extracted {len(sections)} sections from XML")
    return sections

def parse_mapping_details(mapping_xml: str) -> Dict[str, Any]:
    """
    Parse MAPPING XML to extract critical details about data flow
    This helps Claude understand the transformation logic
    """
    details = {
        'mapping_name': '',
        'transformations': [],
        'instances': [],
        'connectors': [],
        'sources': [],
        'targets': []
    }
    
    # Extract mapping name
    mapping_match = re.search(r'<MAPPING\s+[^>]*NAME\s*=\s*"([^"]+)"', mapping_xml)
    if mapping_match:
        details['mapping_name'] = mapping_match.group(1)
    
    # Extract transformation types and names
    trans_pattern = r'<TRANSFORMATION\s+[^>]*NAME\s*=\s*"([^"]+)"[^>]*TYPE\s*=\s*"([^"]+)"'
    for match in re.finditer(trans_pattern, mapping_xml):
        details['transformations'].append({
            'name': match.group(1),
            'type': match.group(2)
        })
    
    # Extract instances (actual usage of transformations)
    inst_pattern = r'<INSTANCE\s+[^>]*NAME\s*=\s*"([^"]+)"[^>]*TRANSFORMATION_NAME\s*=\s*"([^"]+)"[^>]*TRANSFORMATION_TYPE\s*=\s*"([^"]+)"'
    for match in re.finditer(inst_pattern, mapping_xml):
        details['instances'].append({
            'instance_name': match.group(1),
            'transformation_name': match.group(2),
            'transformation_type': match.group(3)
        })
    
    # Extract connectors (data flow) - ALL of them
    conn_pattern = r'<CONNECTOR\s+[^>]*FROMINSTANCE\s*=\s*"([^"]+)"[^>]*TOINSTANCE\s*=\s*"([^"]+)"'
    for match in re.finditer(conn_pattern, mapping_xml):
        details['connectors'].append({
            'from': match.group(1),
            'to': match.group(2)
        })
    
    return details


def create_chunks(xml_content: str, file_name: str) -> List[Tuple[str, int, int]]:
    """
    Create intelligent chunks that preserve logical units
    Key improvement: Keep complete MAPPING and WORKFLOW blocks intact
    """
    estimated_tokens = estimate_token_count(xml_content)
    
    if estimated_tokens < config.MAX_TOKENS:
        logger.debug(
            f"File {file_name} fits in single chunk ({estimated_tokens} tokens)")
        return [(xml_content, 1, 1)]
    
    logger.info(
        f"Large file detected ({estimated_tokens:,} tokens). "
        f"Creating intelligent chunks..."
    )
    
    # Strategy: Split by major logical units, not by arbitrary sections
    chunks = []
    
    # Extract POWERMART header and REPOSITORY info (keep in first chunk)
    header_match = re.search(
        r'(<\?xml[^>]*>.*?<POWERMART[^>]*>.*?<REPOSITORY[^>]*>)',
        xml_content,
        re.DOTALL
    )
    header = header_match.group(1) if header_match else ""
    
    # Extract FOLDER blocks - each folder is a logical unit
    folder_pattern = r'<FOLDER\s+[^>]*>.*?</FOLDER>'
    folders = list(re.finditer(folder_pattern, xml_content, re.DOTALL))
    
    if not folders:
        # No folders found, fall back to section-based chunking
        logger.warning("No FOLDER tags found, using section-based chunking")
        return create_chunks_original(xml_content, file_name)
    
    logger.info(f"Found {len(folders)} FOLDER blocks")
    
    # Group folders into chunks based on token limits
    current_chunk_folders = []
    current_chunk_tokens = estimate_token_count(header)
    
    for folder_match in folders:
        folder_content = folder_match.group(0)
        folder_tokens = estimate_token_count(folder_content)
        
        # Extract folder name for logging
        folder_name_match = re.search(r'NAME\s*=\s*"([^"]+)"', folder_content)
        folder_name = folder_name_match.group(1) if folder_name_match else "unknown"
        
        # Check if adding this folder would exceed token limit
        if current_chunk_tokens + folder_tokens > config.MAX_TOKENS and current_chunk_folders:
            # Create chunk from accumulated folders
            chunk_content = header + "\n" + "\n".join(current_chunk_folders) + "\n</REPOSITORY>\n</POWERMART>"
            chunks.append((chunk_content, len(chunks) + 1, 0))
            logger.info(f"Created chunk {len(chunks)} with {len(current_chunk_folders)} folders")
            
            # Start new chunk
            current_chunk_folders = [folder_content]
            current_chunk_tokens = estimate_token_count(header) + folder_tokens
        else:
            # Add folder to current chunk
            current_chunk_folders.append(folder_content)
            current_chunk_tokens += folder_tokens
        
        logger.debug(f"Folder '{folder_name}': {folder_tokens:,} tokens")
    
    # Add remaining folders as final chunk
    if current_chunk_folders:
        chunk_content = header + "\n" + "\n".join(current_chunk_folders) + "\n</REPOSITORY>\n</POWERMART>"
        chunks.append((chunk_content, len(chunks) + 1, 0))
        logger.info(f"Created final chunk {len(chunks)} with {len(current_chunk_folders)} folders")
    
    # Handle case where a single folder exceeds token limit
    if not chunks:
        logger.warning(f"Single folder exceeds token limit, splitting by MAPPING blocks")
        return split_large_folder(xml_content, file_name)
    
    # Update chunk numbers
    total_chunks = len(chunks)
    chunks = [(content, idx, total_chunks) for content, idx, _ in chunks]
    
    logger.info(f"Created {total_chunks} folder-based chunks for {file_name}")
    return chunks


def split_large_folder(xml_content: str, file_name: str) -> List[Tuple[str, int, int]]:
    """
    Split a very large folder by MAPPING and WORKFLOW blocks
    This is a fallback when a single folder exceeds token limits
    """
    chunks = []
    
    # Extract header
    header_match = re.search(
        r'(<\?xml[^>]*>.*?<POWERMART[^>]*>.*?<REPOSITORY[^>]*>.*?<FOLDER[^>]*>)',
        xml_content,
        re.DOTALL
    )
    header = header_match.group(1) if header_match else ""
    
    # Extract footer
    footer = "</FOLDER>\n</REPOSITORY>\n</POWERMART>"
    
    # Extract major blocks (SOURCE, TARGET, MAPPING, WORKFLOW)
    major_blocks = []
    
    # Pattern for major block types
    block_patterns = [
        (r'<SOURCE\s+[^>]*>.*?</SOURCE>', 'SOURCE'),
        (r'<TARGET\s+[^>]*>.*?</TARGET>', 'TARGET'),
        (r'<MAPPING\s+[^>]*>.*?</MAPPING>', 'MAPPING'),
        (r'<WORKFLOW\s+[^>]*>.*?</WORKFLOW>', 'WORKFLOW'),
        (r'<SHORTCUT\s+[^>]*/>|<SHORTCUT\s+[^>]*>.*?</SHORTCUT>', 'SHORTCUT'),
    ]
    
    for pattern, block_type in block_patterns:
        for match in re.finditer(pattern, xml_content, re.DOTALL):
            major_blocks.append({
                'content': match.group(0),
                'type': block_type,
                'start': match.start(),
                'tokens': estimate_token_count(match.group(0))
            })
    
    # Sort by position in document
    major_blocks.sort(key=lambda x: x['start'])
    
    logger.info(f"Found {len(major_blocks)} major blocks in large folder")
    
    # Group blocks into chunks
    current_chunk_blocks = []
    current_chunk_tokens = estimate_token_count(header)
    
    for block in major_blocks:
        if current_chunk_tokens + block['tokens'] > config.MAX_TOKENS and current_chunk_blocks:
            # Create chunk
            chunk_content = header + "\n" + "\n".join([b['content'] for b in current_chunk_blocks]) + "\n" + footer
            chunks.append((chunk_content, len(chunks) + 1, 0))
            
            # Start new chunk
            current_chunk_blocks = [block]
            current_chunk_tokens = estimate_token_count(header) + block['tokens']
        else:
            current_chunk_blocks.append(block)
            current_chunk_tokens += block['tokens']
    
    # Add remaining blocks
    if current_chunk_blocks:
        chunk_content = header + "\n" + "\n".join([b['content'] for b in current_chunk_blocks]) + "\n" + footer
        chunks.append((chunk_content, len(chunks) + 1, 0))
    
    # Update chunk numbers
    total_chunks = len(chunks)
    chunks = [(content, idx, total_chunks) for content, idx, _ in chunks]
    
    logger.info(f"Split large folder into {total_chunks} chunks")
    return chunks


def create_chunks_original(xml_content: str, file_name: str) -> List[Tuple[str, int, int]]:
    """
    Original chunking logic as fallback
    """
    sections = extract_xml_sections(xml_content)
    chunks = []
    current_chunk = ""
    current_chunk_sections = []
    
    # Extract wrapper elements
    powermart_header = ""
    powermart_footer = "</REPOSITORY>\n</POWERMART>"
    
    header_match = re.search(
        r'(<\?xml[^>]*>.*?<POWERMART[^>]*>.*?<REPOSITORY[^>]*>)',
        xml_content,
        re.DOTALL
    )
    if header_match:
        powermart_header = header_match.group(1)
    
    for section_content, section_type in sections:
        section_tokens = estimate_token_count(section_content)
        current_tokens = estimate_token_count(current_chunk)
        
        if current_tokens + section_tokens > config.MAX_TOKENS and current_chunk:
            wrapped_chunk = (
                f"{powermart_header}\n"
                f"<!-- Chunk containing: {', '.join(current_chunk_sections)} -->\n"
                f"{current_chunk}\n"
                f"{powermart_footer}"
            )
            chunks.append((wrapped_chunk, len(chunks) + 1, 0))
            current_chunk = ""
            current_chunk_sections = []
        
        current_chunk += section_content + "\n"
        current_chunk_sections.append(section_type)
        
        if section_tokens > config.MAX_TOKENS:
            logger.warning(
                f"Single {section_type} section exceeds token limit. "
                f"Splitting by size..."
            )
            section_parts = [
                section_content[i:i+config.CHUNK_SIZE]
                for i in range(0, len(section_content), config.CHUNK_SIZE)
            ]
            for part in section_parts:
                wrapped_part = (
                    f"{powermart_header}\n"
                    f"<!-- Partial {section_type} chunk -->\n"
                    f"{part}\n"
                    f"{powermart_footer}"
                )
                chunks.append((wrapped_part, len(chunks) + 1, 0))
            current_chunk = ""
            current_chunk_sections = []
    
    if current_chunk:
        wrapped_chunk = (
            f"{powermart_header}\n"
            f"<!-- Chunk containing: {', '.join(current_chunk_sections)} -->\n"
            f"{current_chunk}\n"
            f"{powermart_footer}"
        )
        chunks.append((wrapped_chunk, len(chunks) + 1, 0))
    
    total_chunks = len(chunks)
    chunks = [(content, idx, total_chunks) for content, idx, _ in chunks]
    
    logger.info(f"Created {total_chunks} section-based chunks for {file_name}")
    return chunks

# ============================================================================
# PROMPT GENERATION - ENHANCED VERSION
# ============================================================================

def create_conversion_prompt(
    xml_content: str,
    file_name: str,
    project_name: str = None,
    chunk_info: Tuple[int, int] = None,
    is_combined: bool = False
) -> str:
    """
    Enhanced prompt with explicit transformation-to-model mapping - GENERIC VERSION
    """
    
    chunk_context = ""
    if chunk_info:
        chunk_idx, total_chunks = chunk_info
        chunk_context = (
            f"\nNOTE: This is chunk {chunk_idx} of {total_chunks} "
            f"from file {file_name}."
        )
    
    project_context = ""
    if project_name:
        project_context = f"\nPROJECT NAME: {project_name}"
        if is_combined:
            project_context += (
                f"\nNOTE: This is part of a combined DBT project "
                f"'{project_name}' containing multiple workflows."
            )
    
    # Extract detailed mapping analysis
    mapping_analysis = ""
    transformation_flow = ""
    mapping_sections = re.findall(r'<MAPPING\s+[^>]*>.*?</MAPPING>', xml_content, re.DOTALL)
    
    if mapping_sections:
        mapping_analysis = "\n\n=== DETAILED MAPPING ANALYSIS ===\n"
        for idx, mapping_xml in enumerate(mapping_sections[:3], 1):
            details = parse_mapping_details(mapping_xml)
            if details['mapping_name']:
                mapping_analysis += f"\nMapping {idx}: {details['mapping_name']}\n"
                mapping_analysis += f"Transformations found: {len(details['transformations'])}\n"
                
                # List all transformations with types
                if details['transformations']:
                    mapping_analysis += "Transformation breakdown:\n"
                    for trans in details['transformations']:
                        mapping_analysis += f"  - {trans['name']} (Type: {trans['type']})\n"
                
                # Show complete data flow
                if details['connectors']:
                    mapping_analysis += f"\nData flow ({len(details['connectors'])} steps):\n"
                    for conn in details['connectors']:
                        mapping_analysis += f"  {conn['from']} --> {conn['to']}\n"
                    
                    # Build transformation flow for the prompt
                    transformation_flow = "\n\n=== REQUIRED MODEL STRUCTURE ===\n"
                    transformation_flow += "Based on the CONNECTOR tags, you MUST create these models:\n\n"
                    
                    # Identify each step in the flow
                    for i, conn in enumerate(details['connectors'], 1):
                        from_inst = conn['from']
                        to_inst = conn['to']
                        
                        # Determine model type for each step
                        if 'sq_' in from_inst.lower() or 'source' in from_inst.lower():
                            transformation_flow += f"Step {i}: Staging model for {from_inst}\n"
                            transformation_flow += f"  -> File: models/staging/stg_{from_inst.lower()}.sql\n\n"
                        
                        if any(prefix in to_inst.lower() for prefix in ['exp_', 'expression', 'filter', 'router']):
                            transformation_flow += f"Step {i}: Intermediate model for transformation {to_inst}\n"
                            transformation_flow += f"  -> File: models/intermediate/int_{to_inst.lower()}.sql\n"
                            transformation_flow += f"  -> This model MUST reference the previous step using ref()\n\n"
                        
                        elif any(prefix in to_inst.lower() for prefix in ['lkp_', 'lookup', 'jnr_', 'joiner']):
                            transformation_flow += f"Step {i}: Intermediate model for {to_inst}\n"
                            transformation_flow += f"  -> File: models/intermediate/int_{to_inst.lower()}.sql\n"
                            transformation_flow += f"  -> This model MUST implement the join/lookup logic\n\n"
                        
                        elif any(prefix in to_inst.lower() for prefix in ['agg_', 'aggregator', 'rank', 'sorter']):
                            transformation_flow += f"Step {i}: Intermediate model for {to_inst}\n"
                            transformation_flow += f"  -> File: models/intermediate/int_{to_inst.lower()}.sql\n"
                            transformation_flow += f"  -> This model MUST implement aggregation/ranking\n\n"
                    
                    # Final mart model
                    last_connector = details['connectors'][-1] if details['connectors'] else None
                    if last_connector:
                        transformation_flow += f"FINAL STEP: Mart model for target {last_connector['to']}\n"
                        transformation_flow += f"  -> File: models/marts/fct_{last_connector['to'].lower()}.sql or dim_{last_connector['to'].lower()}.sql\n"
                        transformation_flow += f"  -> This MUST be the final model in the chain\n"
                        transformation_flow += f"  -> It MUST reference the last intermediate model\n\n"
    
    prompt = f"""
You are an expert data engineer specializing in migrating Informatica PowerCenter workflows to modern dbt projects.

TASK: Convert the provided Informatica XML workflow into a complete, production-ready dbt project structure.

INPUT FILE: {file_name}{project_context}{chunk_context}{mapping_analysis}{transformation_flow}

=== CRITICAL REQUIREMENTS ===

1. **EVERY TRANSFORMATION MUST BECOME A MODEL**
   - Source Qualifier -> Staging model (stg_*)
   - Expression -> Intermediate model (int_expression_*)
   - Filter -> Intermediate model (int_filter_*)
   - Lookup -> Intermediate model (int_lookup_*)
   - Joiner -> Intermediate model (int_join_*)
   - Aggregator -> Intermediate model (int_agg_*)
   - Any other transformation -> Intermediate model (int_*)
   - Target -> Mart model (fct_* or dim_*)

2. **FOLLOW CONNECTOR FLOW EXACTLY**
   - CONNECTOR tags show: A -> B -> C -> D
   - Create models: stg_a -> int_b -> int_c -> fct_d
   - Each model references the previous one using ref()
   
3. **MANDATORY OUTPUT FOR EVERY WORKFLOW**
   - At least 1 staging model (for each SOURCE)
   - At least 1 intermediate model (for each TRANSFORMATION)
   - Exactly 1 mart model (for each TARGET)
   - schema.yml files for ALL layers
   - sources.yml for staging
   - Macros for repeated logic
   
=== DETAILED CONVERSION RULES ===

**STEP 1: Analyze XML Structure**
1. Find all <SOURCE> tags -> create staging models
2. Find all <TRANSFORMATION> tags -> create intermediate models
3. Find all <CONNECTOR> tags -> determine model dependencies
4. Find all <TARGET> tags -> create mart models

**STEP 2: Source to Staging Mapping**

For EACH <SOURCE> in the XML:
```xml
<SOURCE NAME="[ANY_SOURCE_NAME]" DBDNAME="[SCHEMA]" OWNERNAME="[OWNER]">
  <SOURCEFIELD NAME="[FIELD1]" DATATYPE="[TYPE1]" KEYTYPE="PRIMARY KEY"/>
  <SOURCEFIELD NAME="[FIELD2]" DATATYPE="[TYPE2]"/>
</SOURCE>
```

You MUST create:

FILE: models/staging/stg_[source_name_lowercase].sql
```sql
-- Staging: [SOURCE_NAME]
-- Source: [SCHEMA].[SOURCE_NAME]
-- Raw data from source system

select
    [field1],
    [field2],
    [field3]
    -- Include ALL fields from source
from {{{{ source('[schema_name]', '[source_table_name]') }}}}
```

**STEP 3: Transformation to Intermediate Mapping**

For Source Qualifier transformation:
```xml
<TRANSFORMATION NAME="[SQ_NAME]" TYPE="Source Qualifier">
```

Create:
FILE: models/intermediate/int_[sq_name_lowercase].sql
```sql
-- Source Qualifier: [SQ_NAME]
-- Applies source filtering and column selection

select 
    [column1],
    [column2],
    [column3]
    -- Select only needed columns
from {{{{ ref('stg_[source_name]') }}}}
where [filter_conditions_if_any]
```

For Expression transformation:
```xml
<TRANSFORMATION NAME="[EXP_NAME]" TYPE="Expression">
  <TRANSFORMFIELD NAME="[OUTPUT_FIELD]" EXPRESSION="[EXPRESSION_LOGIC]"/>
</TRANSFORMATION>
```

Create:
FILE: models/intermediate/int_[exp_name_lowercase].sql
```sql
-- Expression: [EXP_NAME]
-- Applies business logic transformations

select
    *,  -- Carry forward all existing columns
    [expression_logic] as [output_field],
    [another_expression] as [another_field]
    -- Add calculated columns based on TRANSFORMFIELD expressions
from {{{{ ref('[previous_model_name]') }}}}
```

For Lookup transformation:
```xml
<TRANSFORMATION NAME="[LKP_NAME]" TYPE="Lookup Procedure">
```

Create:
FILE: models/intermediate/int_[lkp_name_lowercase].sql
```sql
-- Lookup: [LKP_NAME]
-- Enriches data with lookup values

select
    a.*,
    b.[lookup_column1],
    b.[lookup_column2]
from {{{{ ref('[previous_model_name]') }}}} a
left join {{{{ ref('stg_[lookup_source]') }}}} b
    on a.[join_key] = b.[lookup_key]
```

For Filter transformation:
```xml
<TRANSFORMATION NAME="[FLT_NAME]" TYPE="Filter">
```

Create:
FILE: models/intermediate/int_[flt_name_lowercase].sql
```sql
-- Filter: [FLT_NAME]
-- Filters rows based on conditions

select *
from {{{{ ref('[previous_model_name]') }}}}
where [filter_condition]
```

For Joiner transformation:
```xml
<TRANSFORMATION NAME="[JNR_NAME]" TYPE="Joiner">
```

Create:
FILE: models/intermediate/int_[jnr_name_lowercase].sql
```sql
-- Joiner: [JNR_NAME]
-- Joins multiple data streams

select
    a.*,
    b.[columns_from_second_stream]
from {{{{ ref('[first_input_model]') }}}} a
[join_type] join {{{{ ref('[second_input_model]') }}}} b
    on a.[key] = b.[key]
```

For Aggregator transformation:
```xml
<TRANSFORMATION NAME="[AGG_NAME]" TYPE="Aggregator">
```

Create:
FILE: models/intermediate/int_[agg_name_lowercase].sql
```sql
-- Aggregator: [AGG_NAME]
-- Aggregates data

select
    [group_by_columns],
    count(*) as record_count,
    sum([measure_column]) as total_[measure],
    avg([measure_column]) as avg_[measure]
from {{{{ ref('[previous_model_name]') }}}}
group by [group_by_columns]
```

**STEP 4: Target to Mart Mapping**

For database table target:
```xml
<TARGET NAME="[TARGET_NAME]" DATABASETYPE="Oracle">
</TARGET>
```

Create:
FILE: models/marts/fct_[target_name_lowercase].sql (for fact tables)
OR
FILE: models/marts/dim_[target_name_lowercase].sql (for dimension tables)

```sql
-- Mart: [TARGET_NAME]
-- Final analytical data ready for consumption
-- Data flow: [list all models in the chain]

{{{{
    config(
        materialized='table',
        tags=['[domain]', 'mart']
    )
}}}}

select
    [all_final_columns],
    current_timestamp() as loaded_at
from {{{{ ref('[last_intermediate_model]') }}}}
```

For flat file target:
```xml
<TARGET NAME="[FF_TARGET_NAME]" DATABASETYPE="Flat File">
</TARGET>
```

Create:
FILE: models/marts/fct_[target_name_lowercase].sql
```sql
-- Mart: [FF_TARGET_NAME] (Flat file export)
-- Prepares data for external file export
-- Note: Actual file writing happens outside DBT

{{{{
    config(
        materialized='table',
        tags=['export', 'flatfile']
    )
}}}}

select *
from {{{{ ref('[last_intermediate_model]') }}}}
```

**STEP 5: Create ALL Required YAML Files**

FILE: dbt_project.yml
```yaml
name: '{project_name or file_name.replace(".xml", "").replace(".XML", "").lower()}'
version: '1.0.0'
config-version: 2

models:
  {project_name or file_name.replace(".xml", "").replace(".XML", "").lower()}:
    staging:
      +materialized: view
      +tags: ['staging']
    intermediate:
      +materialized: view
      +tags: ['intermediate']
    marts:
      +materialized: table
      +tags: ['mart']
```

FILE: models/staging/sources.yml
```yaml
version: 2

sources:
  - name: [source_system_name]
    description: "Source system: [description]"
    database: [database_name]
    schema: [schema_name]
    tables:
      - name: [table_name]
        description: "Source table: [table_name]"
        columns:
          - name: [primary_key_column]
            description: "Primary key"
            tests:
              - unique
              - not_null
          - name: [other_columns]
            description: "Column description"
```

FILE: models/staging/schema.yml
```yaml
version: 2

models:
  - name: stg_[table_name]
    description: "Staging model for [source_table_name]"
    columns:
      - name: [primary_key]
        description: "Primary key column"
        tests:
          - unique
          - not_null
      - name: [other_columns]
        description: "Column description"
```

FILE: models/intermediate/schema.yml
```yaml
version: 2

models:
  - name: int_[transformation_name]
    description: "Intermediate model: [transformation description]"
    columns:
      - name: [key_columns]
        description: "Key column description"
        tests:
          - not_null
```

FILE: models/marts/schema.yml
```yaml
version: 2

models:
  - name: fct_[target_name]
    description: "Fact/Dim table: [target description]"
    columns:
      - name: [primary_key]
        description: "Primary key"
        tests:
          - unique
          - not_null
      - name: [measure_columns]
        description: "Measure/attribute description"
```

**STEP 6: Create Macros for Reusable Logic**

If you see repeated transformation patterns (like date formatting, string manipulation), create macros:

FILE: macros/[macro_name].sql
```sql
{{% macro [macro_name]([parameters]) %}}
    -- Macro description: [what it does]
    [SQL transformation logic]
{{% endmacro %}}
```

Then use in models:
```sql
select
    {{{{ [macro_name]([column_name]) }}}} as [output_column]
from {{{{ ref('[source_model]') }}}}
```

=== VALIDATION CHECKLIST ===

Before submitting, verify you created:
☐ Staging model (stg_*.sql) for EVERY <SOURCE> tag
☐ Intermediate model (int_*.sql) for EVERY <TRANSFORMATION> tag  
☐ Mart model (fct_*.sql or dim_*.sql) for EVERY <TARGET> tag
☐ sources.yml with ALL source tables defined
☐ schema.yml in staging/ with ALL staging models documented
☐ schema.yml in intermediate/ with ALL intermediate models documented
☐ schema.yml in marts/ with ALL mart models documented
☐ dbt_project.yml with correct project configuration
☐ Macros for any repeated transformation logic
☐ Model dependencies follow CONNECTOR flow exactly
☐ Each model uses ref() to reference the correct previous model
☐ Primary keys have unique and not_null tests

=== COMMON TRANSFORMATION PATTERNS ===

**Pattern 1: Simple Pass-Through**
If a transformation just passes data through, still create the model:
```sql
select * from {{{{ ref('[previous_model]') }}}}
```

**Pattern 2: Column Renaming**
```sql
select
    [old_column_name] as [new_column_name],
    [other_columns]
from {{{{ ref('[previous_model]') }}}}
```

**Pattern 3: Type Casting**
```sql
select
    cast([column] as [new_type]) as [column],
    [other_columns]
from {{{{ ref('[previous_model]') }}}}
```

**Pattern 4: Conditional Logic**
```sql
select
    *,
    case
        when [condition1] then [value1]
        when [condition2] then [value2]
        else [default_value]
    end as [new_column]
from {{{{ ref('[previous_model]') }}}}
```

=== OUTPUT FORMAT ===

Structure your response with clear file headers:

FILE: dbt_project.yml
```yaml
[content]
```

FILE: models/staging/sources.yml
```yaml
[content]
```

FILE: models/staging/schema.yml
```yaml
[content]
```

FILE: models/staging/stg_[name].sql
```sql
[content]
```

FILE: models/intermediate/schema.yml
```yaml
[content]
```

FILE: models/intermediate/int_[name].sql
```sql
[content]
```

FILE: models/marts/schema.yml
```yaml
[content]
```

FILE: models/marts/fct_[name].sql
```sql
[content]
```

FILE: macros/[name].sql (if needed)
```sql
[content]
```

=== INFORMATICA XML WORKFLOW ===

```xml
{xml_content}
```

**FINAL REMINDERS:**
1. Count TRANSFORMATION tags in XML = number of intermediate models you must create
2. Count TARGET tags in XML = number of mart models you must create  
3. Count SOURCE tags in XML = number of staging models you must create
4. Create schema.yml for EVERY layer (staging, intermediate, marts)
5. Follow CONNECTOR flow: each model refs the previous one in the chain
6. Use generic names based on XML, not hardcoded names
7. Every target MUST have a mart model - no exceptions!

Convert this workflow to a complete dbt project NOW.
"""
    return prompt

# ============================================================================
# CLAUDE API CALLS
# ============================================================================

@rate_limit(calls_per_minute=config.RATE_LIMIT_CALLS_PER_MINUTE)
@retry_on_failure(max_retries=config.MAX_RETRIES, delay=config.RETRY_DELAY)
def call_claude_api(prompt: str) -> str:
    """Call Claude API with rate limiting and retries"""
    try:
        logger.debug(f"Calling Claude API with {len(prompt)} character prompt")
        response = complete(config.CLAUDE_MODEL, prompt)
        
        if not response:
            raise ClaudeAPIError("Empty response from Claude API")
        
        return response
        
    except Exception as e:
        logger.error(f"Claude API call failed: {e}")
        raise ClaudeAPIError(f"Claude API error: {str(e)}")

# ============================================================================
# VALIDATION AND PARSING
# ============================================================================

def validate_dbt_sql(sql_content: str) -> Tuple[bool, Optional[str]]:
    """Validate DBT SQL content"""
    try:
        # Check for required DBT patterns
        has_ref_or_source = bool(
            re.search(r'{{\s*(ref|source)\s*\(', sql_content)
        )
        if not has_ref_or_source:
            return False, "No ref() or source() found in SQL"
        
        # Check for balanced Jinja braces
        open_count = sql_content.count('{{')
        close_count = sql_content.count('}}')
        
        if open_count != close_count:
            return False, f"Unbalanced Jinja braces: {open_count} {{ vs {close_count} }}"
        
        return True, None
    
    except Exception as e:
        return False, f"Validation error: {str(e)}"

def validate_yaml_content(yaml_content: str) -> Tuple[bool, Optional[str]]:
    """Validate YAML content structure"""
    try:
        # Basic YAML structure checks
        if not yaml_content.strip():
            return False, "Empty YAML content"
        
        # Check for common YAML errors
        if yaml_content.count(':') == 0:
            return False, "No key-value pairs found"
        
        # Check indentation consistency
        lines = yaml_content.split('\n')
        indents = set()
        for line in lines:
            if line.strip() and not line.strip().startswith('#'):
                indent = len(line) - len(line.lstrip())
                if indent > 0:
                    indents.add(indent)
        
        if indents and min(indents) not in [2, 4]:
            return False, "Inconsistent indentation (should be 2 or 4 spaces)"
        
        return True, None
    
    except Exception as e:
        return False, f"Validation error: {str(e)}"

def parse_claude_response(response: str) -> Dict[str, str]:
    """Parse Claude response to extract individual DBT files"""
    files = {}
    
    try:
        # Primary pattern: FILE: path
        file_pattern = r'FILE:\s*([^\n]+)\n```(?:sql|yaml|yml)?\n(.*?)```'
        matches = re.findall(file_pattern, response, re.DOTALL | re.MULTILINE)
        
        for file_path, content in matches:
            clean_path = file_path.strip()
            clean_content = content.strip()
            
            if clean_path and clean_content:
                # Validate based on file type
                if clean_path.endswith('.sql'):
                    is_valid, error = validate_dbt_sql(clean_content)
                    if not is_valid:
                        logger.warning(
                            f"SQL validation failed for {clean_path}: {error}"
                        )
                elif clean_path.endswith(('.yml', '.yaml')):
                    is_valid, error = validate_yaml_content(clean_content)
                    if not is_valid:
                        logger.warning(
                            f"YAML validation failed for {clean_path}: {error}"
                        )
                
                files[clean_path] = clean_content
        
        # Alternative pattern if primary doesn't work
        if not files:
            alt_pattern = r'([^\n]*\.(?:sql|yml|yaml))\n```(?:sql|yaml|yml)?\n(.*?)```'
            alt_matches = re.findall(alt_pattern, response, re.DOTALL)
            
            for file_path, content in alt_matches:
                clean_path = file_path.strip()
                clean_content = content.strip()
                
                if clean_path and clean_content:
                    files[clean_path] = clean_content
        
        if not files:
            logger.warning(
                "No structured files found in Claude response, saving raw response"
            )
            files['claude_response.txt'] = response
        
        logger.info(f"Parsed {len(files)} files from Claude response")
        return files
        
    except Exception as e:
        logger.error(f"Error parsing Claude response: {e}")
        return {'claude_response.txt': response}

def convert_xml_to_dbt(
    session,
    xml_content: str,
    file_name: str,
    project_name: str = None,
    retry_count: int = 0,
    is_combined: bool = False
) -> Dict[str, Any]:
    """Convert XML to DBT using Claude with chunking support"""
    try:
        estimated_tokens = estimate_token_count(xml_content)
        
        if estimated_tokens > config.MAX_TOKENS:
            logger.info(
                f"File {file_name} requires chunking "
                f"({estimated_tokens:,} estimated tokens)"
            )
            return convert_large_xml_to_dbt(
                session, xml_content, file_name, project_name, is_combined
            )
        else:
            logger.info(
                f"Converting {file_name} using {config.CLAUDE_MODEL} "
                f"(attempt {retry_count + 1})"
            )
            prompt = create_conversion_prompt(
                xml_content, file_name, project_name, None, is_combined
            )
            
            with track_time(f"Claude conversion for {file_name}"):
                start_time = time.time()
                response = call_claude_api(prompt)
                conversion_time = time.time() - start_time
            
            dbt_files = parse_claude_response(response)
            
            if not dbt_files:
                raise ConversionError("No DBT files generated from Claude response")
            
            logger.info(f"Generated {len(dbt_files)} DBT files for {file_name}")
            
            return {
                'status': 'success',
                'files': dbt_files,
                'processing_time': conversion_time,
                'file_count': len(dbt_files),
                'chunks_processed': 1
            }
    
    except ClaudeAPIError as e:
        if "token" in str(e).lower() and retry_count == 0:
            logger.warning("Token limit exceeded, retrying with chunking")
            return convert_large_xml_to_dbt(
                session, xml_content, file_name, project_name, is_combined
            )
        
        error_msg = f"Conversion failed: {str(e)}"
        logger.error(error_msg)
        logger.debug(traceback.format_exc())
        
        return {
            'status': 'failed',
            'error': error_msg,
            'processing_time': 0
        }
    
    except Exception as e:
        error_msg = f"Unexpected error during conversion: {str(e)}"
        logger.error(error_msg)
        logger.debug(traceback.format_exc())
        
        return {
            'status': 'failed',
            'error': error_msg,
            'processing_time': 0
        }

def convert_large_xml_to_dbt(
    session,
    xml_content: str,
    file_name: str,
    project_name: str = None,
    is_combined: bool = False
) -> Dict[str, Any]:
    """Convert large XML files using intelligent chunking"""
    try:
        logger.info(f"Starting chunked conversion for {file_name}")
        
        chunks = create_chunks(xml_content, file_name)
        all_dbt_files = {}
        total_time = 0
        successful_chunks = 0
        
        for chunk_content, chunk_idx, total_chunks in chunks:
            logger.info(
                f"Processing chunk {chunk_idx}/{total_chunks} of {file_name}")
            
            prompt = create_conversion_prompt(
                chunk_content, file_name, project_name,
                (chunk_idx, total_chunks), is_combined
            )
            
            try:
                with track_time(f"Chunk {chunk_idx}/{total_chunks}"):
                    start_time = time.time()
                    response = call_claude_api(prompt)
                    chunk_time = time.time() - start_time
                    total_time += chunk_time
                
                chunk_files = parse_claude_response(response)
                
                for file_path, content in chunk_files.items():
                    if file_path.endswith('.sql') and total_chunks > 1:
                        base_name = file_path.rsplit('.', 1)[0]
                        chunk_file_path = f"{base_name}_chunk{chunk_idx}.sql"
                        all_dbt_files[chunk_file_path] = content
                    elif file_path.endswith(('.yml', '.yaml')):
                        if file_path in all_dbt_files:
                            all_dbt_files[file_path] += (
                                f"\n# --- Chunk {chunk_idx} ---\n{content}"
                            )
                        else:
                            all_dbt_files[file_path] = content
                    else:
                        all_dbt_files[file_path] = content
                
                successful_chunks += 1
                logger.info(f"Chunk {chunk_idx} processed successfully")
                
            except Exception as e:
                logger.error(f"Error processing chunk {chunk_idx}: {e}")
                continue
            
            if chunk_idx < total_chunks:
                time.sleep(1)
        
        if not all_dbt_files:
            raise ConversionError("No DBT files generated from any chunks")
        
        if total_chunks > 1:
            summary = f"""# Chunking Summary for {file_name}
- Total chunks: {total_chunks}
- Successful chunks: {successful_chunks}
- Total processing time: {total_time:.2f} seconds
- Files generated: {len(all_dbt_files)}

Note: SQL files have been split by chunk. Review and merge as needed.
"""
            all_dbt_files['CHUNKING_NOTES.md'] = summary
        
        logger.info(
            f"Chunked conversion completed: {successful_chunks}/{total_chunks} "
            f"chunks successful, {len(all_dbt_files)} files generated"
        )
        
        return {
            'status': 'success',
            'files': all_dbt_files,
            'processing_time': total_time,
            'file_count': len(all_dbt_files),
            'chunks_processed': total_chunks
        }
        
    except Exception as e:
        error_msg = f"Chunked conversion failed: {str(e)}"
        logger.error(error_msg)
        logger.debug(traceback.format_exc())
        
        return {
            'status': 'failed',
            'error': error_msg,
            'processing_time': 0
        }

# ============================================================================
# FILE COMBINATION AND REFERENCE FIXING
# ============================================================================

def fix_refs_in_content(
    content: str,
    workflow_prefix: str,
    all_model_names: set
) -> str:
    """Fix ref() statements to include workflow prefix or match existing model names"""
    ref_pattern = r"{{\s*ref\(['\"]([^'\"]+)['\"]\)\s*}}"
    
    def replace_ref(match):
        model_ref = match.group(1)
        
        if model_ref.startswith(workflow_prefix + '_'):
            return match.group(0)
        
        prefixed_ref = f"{workflow_prefix}_{model_ref}"
        
        if prefixed_ref in all_model_names:
            return f"{{{{ ref('{prefixed_ref}') }}}}"
        
        for model_name in all_model_names:
            if model_name.endswith('_' + model_ref) or model_name == model_ref:
                return f"{{{{ ref('{model_name}') }}}}"
        
        return f"{{{{ ref('{prefixed_ref}') }}}}"
    
    return re.sub(ref_pattern, replace_ref, content)

def combine_dbt_files_for_project(
    all_files_list: List[Dict[str, str]],
    project_name: str,
    file_names: List[str]
) -> Dict[str, str]:
    """Combine multiple DBT file sets into a single project with fixed references"""
    combined_files = {}
    model_mapping = {}
    all_model_names = set()
    
    # First pass: collect all model names
    for files, source_file in zip(all_files_list, file_names):
        workflow_prefix = (
            source_file.replace('.xml', '')
            .replace('.XML', '')
            .replace(' ', '_')
            .lower()
        )
        
        for file_path in files.keys():
            if file_path.endswith('.sql') and file_path.startswith('models/'):
                parts = file_path.split('/')
                if len(parts) >= 3:
                    model_name = parts[-1].replace('.sql', '')
                    prefixed_name = f"{workflow_prefix}_{model_name}"
                    all_model_names.add(prefixed_name)
    
    logger.debug(f"Collected {len(all_model_names)} model names across all files")
    
    # Second pass: process and combine files
    for idx, (files, source_file) in enumerate(zip(all_files_list, file_names)):
        workflow_prefix = (
            source_file.replace('.xml', '')
            .replace('.XML', '')
            .replace(' ', '_')
            .lower()
        )
        
        for file_path, content in files.items():
            if file_path == 'dbt_project.yml':
                if 'dbt_project.yml' not in combined_files:
                    updated_content = re.sub(
                        r"name:\s*['\"]?\w+['\"]?",
                        f"name: '{project_name}'",
                        content
                    )
                    
                    if idx == 0 and len(file_names) > 1:
                        workflows_comment = (
                            "\n# This dbt project combines multiple workflows:\n"
                            + "\n".join(
                                f"# - {name.replace('.xml', '').replace('.XML', '')}"
                                for name in file_names
                            )
                        )
                        lines = updated_content.split('\n')
                        for i, line in enumerate(lines):
                            if 'config-version:' in line:
                                lines.insert(i + 1, workflows_comment)
                                break
                        updated_content = '\n'.join(lines)
                    
                    combined_files[file_path] = updated_content
                    
            elif file_path.endswith('.sql'):
                fixed_content = fix_refs_in_content(
                    content, workflow_prefix, all_model_names
                )
                
                if file_path.startswith('models/'):
                    parts = file_path.split('/')
                    if len(parts) >= 3:
                        model_name = parts[-1]
                        new_name = f"{workflow_prefix}_{model_name}"
                        new_path = f"{'/'.join(parts[:-1])}/{new_name}"
                        combined_files[new_path] = (
                            f"-- Source: {source_file}\n{fixed_content}"
                        )
                        model_mapping[new_path] = source_file
                else:
                    combined_files[file_path] = fixed_content
                    
            elif file_path.endswith(('.yml', '.yaml')):
                if file_path == 'dbt_project.yml':
                    pass
                elif 'sources.yml' in file_path or 'schema.yml' in file_path:
                    if file_path in combined_files:
                        content_to_add = content
                        if content.startswith('version:'):
                            lines = content.split('\n')
                            content_to_add = '\n'.join(lines[1:])
                        combined_files[file_path] += (
                            f"\n# --- From {source_file} ---\n{content_to_add}"
                        )
                    else:
                        combined_files[file_path] = f"# From {source_file}\n{content}"
                else:
                    if file_path in combined_files:
                        combined_files[file_path] += (
                            f"\n# --- From {source_file} ---\n{content}"
                        )
                    else:
                        combined_files[file_path] = f"# From {source_file}\n{content}"
            
            else:
                if file_path in combined_files:
                    combined_files[file_path] += (
                        f"\n\n--- From {source_file} ---\n{content}"
                    )
                else:
                    combined_files[file_path] = content
    
    # Add project README
    readme_content = f"""# DBT Project: {project_name}

## Overview
This DBT project was automatically generated from Informatica PowerCenter XML workflows.

## Source Files
This project combines the following workflows:
{chr(10).join(f"- {name}" for name in file_names)}

## Project Structure
- `models/staging/`: Raw data staging models
- `models/intermediate/`: Business logic transformations
- `models/marts/`: Final analytical models

## Model Mapping
Each model is prefixed with its source workflow for clarity.

Generated: {datetime.now().isoformat()}
Converter Version: Production v2.0
"""
    combined_files['README.md'] = readme_content
    
    logger.info(
        f"Combined {len(all_files_list)} file sets into {len(combined_files)} files")
    
    return combined_files

# ============================================================================
# PERSISTENCE FUNCTIONS
# ============================================================================

@retry_on_failure(max_retries=3, delay=2)
def save_dbt_files(
    session,
    dbt_files: Dict[str, str],
    workflow_name: str,
    project_name: str = None
) -> bool:
    """Save DBT files to Snowflake table with project information"""
    try:
        effective_project = project_name or workflow_name
        
        file_records = []
        
        for file_path, content in dbt_files.items():
            record = {
                'project_name': effective_project,
                'workflow_name': workflow_name,
                'file_path': file_path,
                'file_name': file_path.split('/')[-1],
                'file_type': file_path.split('.')[-1] if '.' in file_path else 'txt',
                'content': content,
                'content_length': len(content),
                'created_timestamp': datetime.now().isoformat()
            }
            file_records.append(record)
        
        df = session.create_dataframe(file_records)
        
        with track_time(f"Saving {len(dbt_files)} files to Snowflake"):
            df.write.mode("append").save_as_table(config.OUTPUT_TABLE)
        
        logger.info(
            f"Saved {len(dbt_files)} files to {config.OUTPUT_TABLE} "
            f"for project '{effective_project}'"
        )
        return True
        
    except Exception as e:
        logger.error(f"Error saving DBT files: {e}")
        logger.debug(traceback.format_exc())
        return False

def create_summary_record(
    file_name: str,
    workflow_name: str,
    project_name: str,
    result: Dict[str, Any]
) -> Dict[str, Any]:
    """Create summary record for conversion result"""
    return {
        'source_file': file_name,
        'project_name': project_name,
        'workflow_name': workflow_name,
        'conversion_status': result['status'],
        'files_generated': int(result.get('file_count', 0)),
        'chunks_processed': int(result.get('chunks_processed', 1)),
        'processing_time_seconds': float(result.get('processing_time', 0.0)),
        'error_message': result.get('error', None),
        'converted_timestamp': datetime.now().isoformat()
    }

def save_conversion_summary(session, results: List[Dict[str, Any]]):
    """Save conversion summary to Snowflake"""
    try:
        if not results:
            logger.warning("No results to save in summary")
            return
        
        df = session.create_dataframe(results)
        
        with track_time("Saving conversion summary"):
            df.write.mode("overwrite").save_as_table(config.SUMMARY_TABLE)
        
        logger.info(f"Conversion summary saved to {config.SUMMARY_TABLE}")
        
    except Exception as e:
        logger.error(f"Error saving summary: {e}")
        logger.debug(traceback.format_exc())

# ============================================================================
# BATCH PROCESSING
# ============================================================================

def process_folder_group(
    session,
    folder_name: str,
    file_paths: List[str]
) -> List[Dict[str, Any]]:
    """Process a group of files from the same folder as one DBT project"""
    results = []
    
    project_name = f"dbt_{folder_name.lower().replace(' ', '_').replace('-', '_')}"
    
    logger.info(f"\n{'='*60}")
    logger.info(f"Processing folder: {folder_name}")
    logger.info(f"Project name: {project_name}")
    logger.info(f"Files in folder: {len(file_paths)}")
    logger.info(f"{'='*60}")
    
    if len(file_paths) == 1:
        file_path = file_paths[0]
        file_name = file_path.split('/')[-1]
        
        logger.info(f"Processing single file: {file_name}")
        
        try:
            xml_content = read_xml_content(session, config.INPUT_STAGE, file_path)
            result = convert_xml_to_dbt(
                session, xml_content, file_name, project_name, is_combined=False
            )
            
            if result['status'] == 'success':
                workflow_name = (
                    file_name.replace('.xml', '')
                    .replace('.XML', '')
                    .replace(' ', '_')
                    .lower()
                )
                save_success = save_dbt_files(
                    session, result['files'], workflow_name, project_name
                )
                
                if save_success:
                    logger.info(
                        f"Successfully saved {result['file_count']} DBT files "
                        f"for project '{project_name}'"
                    )
                else:
                    result['status'] = 'partial_success'
                    result['error'] = 'Generated files but failed to save'
                    
        except XMLReadError as e:
            result = {
                'status': 'failed',
                'error': str(e),
                'processing_time': 0
            }
        except Exception as e:
            result = {
                'status': 'failed',
                'error': f"Unexpected error: {str(e)}",
                'processing_time': 0
            }
        
        workflow_name = (
            file_name.replace('.xml', '')
            .replace('.XML', '')
            .replace(' ', '_')
            .lower()
        )
        summary = create_summary_record(file_name, workflow_name, project_name, result)
        results.append(summary)
        
    else:
        logger.info(f"Combining {len(file_paths)} files into single DBT project")
        
        all_files = []
        all_file_names = []
        combined_time = 0
        
        for file_path in file_paths:
            file_name = file_path.split('/')[-1]
            logger.info(f"Processing: {file_name}")
            
            try:
                xml_content = read_xml_content(session, config.INPUT_STAGE, file_path)
                result = convert_xml_to_dbt(
                    session, xml_content, file_name, project_name, is_combined=True
                )
                
                workflow_name = (
                    file_name.replace('.xml', '')
                    .replace('.XML', '')
                    .replace(' ', '_')
                    .lower()
                )
                
                if result['status'] == 'success':
                    all_files.append(result['files'])
                    all_file_names.append(file_name)
                    combined_time += result.get('processing_time', 0)
                
                summary = create_summary_record(
                    file_name, workflow_name, project_name, result
                )
                results.append(summary)
                
            except Exception as e:
                logger.error(f"Failed to process {file_name}: {e}")
                workflow_name = (
                    file_name.replace('.xml', '')
                    .replace('.XML', ''
)
                    .replace(' ', '_')
                    .lower()
                )
                result = {
                    'status': 'failed',
                    'error': str(e),
                    'processing_time': 0
                }
                summary = create_summary_record(
                    file_name, workflow_name, project_name, result
                )
                results.append(summary)
        
        if all_files:
            logger.info(
                f"Combining {len(all_files)} successful conversions "
                f"into project '{project_name}'"
            )
            combined_files = combine_dbt_files_for_project(
                all_files, project_name, all_file_names
            )
            
            save_success = save_dbt_files(
                session, combined_files, f"{project_name}_combined", project_name
            )
            
            if save_success:
                logger.info(
                    f"Successfully created combined DBT project '{project_name}' "
                    f"with {len(combined_files)} files"
                )
    
    return results

def process_files_batch(
    session,
    folder_files: Dict[str, List[str]]
) -> List[Dict[str, Any]]:
    """Process XML files grouped by folder"""
    all_results = []
    total_folders = len(folder_files)
    
    logger.info(f"\nStarting batch processing of {total_folders} folders")
    logger.info(f"Model: {config.CLAUDE_MODEL}, Max Tokens: {config.MAX_TOKENS:,}")
    
    for folder_idx, (folder_name, file_paths) in enumerate(folder_files.items(), 1):
        logger.info(
            f"\n[Folder {folder_idx}/{total_folders}] Processing: {folder_name}")
        
        try:
            folder_results = process_folder_group(session, folder_name, file_paths)
            all_results.extend(folder_results)
        except Exception as e:
            logger.error(f"Error processing folder {folder_name}: {e}")
            logger.debug(traceback.format_exc())
        
        if folder_idx < total_folders:
            logger.debug("Pausing between folders")
            time.sleep(2)
    
    return all_results

# ============================================================================
# REPORTING
# ============================================================================

def print_final_report(results: List[Dict[str, Any]]):
    """Print comprehensive final conversion report"""
    successful = [r for r in results if r['conversion_status'] == 'success']
    failed = [r for r in results if r['conversion_status'] == 'failed']
    
    projects = {}
    for r in results:
        project = r.get('project_name', 'unknown')
        if project not in projects:
            projects[project] = {
                'success': 0,
                'failed': 0,
                'files': 0,
                'chunks': 0,
                'time': 0
            }
        
        if r['conversion_status'] == 'success':
            projects[project]['success'] += 1
            projects[project]['files'] += r.get('files_generated', 0)
            projects[project]['chunks'] += r.get('chunks_processed', 1)
        else:
            projects[project]['failed'] += 1
        
        projects[project]['time'] += r.get('processing_time_seconds', 0)
    
    total_time = sum(r['processing_time_seconds'] for r in results)
    total_files = sum(r['files_generated'] for r in successful)
    total_chunks = sum(r.get('chunks_processed', 1) for r in results)
    
    logger.info("\n" + "="*80)
    logger.info("INFORMATICA TO DBT CONVERSION - FINAL REPORT")
    logger.info("="*80)
    logger.info(f"Total XML Files Processed: {len(results)}")
    logger.info(f"Total DBT Projects Created: {len(projects)}")
    logger.info(f"Successful Conversions: {len(successful)}")
    logger.info(f"Failed Conversions: {len(failed)}")
    logger.info(
        f"Success Rate: {len(successful)/len(results)*100:.1f}%"
        if results else "N/A"
    )
    logger.info(f"Total DBT Files Generated: {total_files}")
    logger.info(f"Total Chunks Processed: {total_chunks}")
    logger.info(f"Total Processing Time: {total_time:.2f} seconds")
    logger.info(
        f"Average Time per File: {total_time/len(results):.2f} seconds"
        if results else "N/A"
    )
    
    logger.info("\nProject Summary:")
    logger.info("-" * 60)
    for project_name, stats in projects.items():
        logger.info(f"\nProject: {project_name}")
        logger.info(
            f"  • Workflows: {stats['success'] + stats['failed']} "
            f"({stats['success']} successful, {stats['failed']} failed)"
        )
        logger.info(f"  • Files generated: {stats['files']}")
        logger.info(f"  • Chunks processed: {stats['chunks']}")
        logger.info(f"  • Processing time: {stats['time']:.2f} seconds")
    
    if successful:
        logger.info("\nSuccessfully Converted Workflows:")
        for r in successful:
            chunks_info = (
                f" ({r.get('chunks_processed', 1)} chunks)"
                if r.get('chunks_processed', 1) > 1 else ""
            )
            logger.info(
                f"  • {r['workflow_name']} → {r['project_name']} "
                f"({r['files_generated']} files{chunks_info})"
            )
    
    if failed:
        logger.info("\nFailed Conversions:")
        for r in failed:
            logger.info(f"  • {r['source_file']}: {r['error_message']}")
    
    logger.info("\nNext Steps:")
    logger.info(f"1. Review generated files in table: {config.OUTPUT_TABLE}")
    logger.info(
        "2. Query by project_name to get all files for a specific DBT project")
    logger.info("3. Download and test DBT projects in your development environment")
    logger.info("4. For chunked files, review and merge SQL models as needed")
    logger.info("5. Validate SQL logic and add custom business rules")
    logger.info("6. Deploy to production DBT environment")
    logger.info("="*80)

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """Main execution function"""
    logger.info("="*80)
    logger.info("Informatica PowerCenter XML to DBT Conversion Script")
    logger.info("Production Version v2.0")
    logger.info("="*80)
    logger.info(f"Using {config.CLAUDE_MODEL} via Snowflake Cortex")
    logger.info(
        "Enhanced with explicit transformation mapping and improved prompting")
    
    logger.info(f"\nConfiguration:")
    logger.info(f"  Input Stage: {config.INPUT_STAGE}")
    logger.info(f"  Output Table: {config.OUTPUT_TABLE}")
    logger.info(f"  Summary Table: {config.SUMMARY_TABLE}")
    logger.info(f"  Claude Model: {config.CLAUDE_MODEL}")
    logger.info(f"  Max Tokens: {config.MAX_TOKENS:,}")
    logger.info(f"  Chunk Size: {config.CHUNK_SIZE:,} characters")
    logger.info(f"  Batch Size: {config.BATCH_SIZE}")
    logger.info(f"  Max Retries: {config.MAX_RETRIES}")
    logger.info(f"  Rate Limit: {config.RATE_LIMIT_CALLS_PER_MINUTE} calls/minute")
    
    try:
        session = get_snowflake_session()
        
        folder_files = list_xml_files_with_folders(session, config.INPUT_STAGE)
        
        if not folder_files:
            logger.warning(f"No XML files found in {config.INPUT_STAGE}")
            logger.info(
                "Please upload your Informatica XML files to the stage and try again.")
            return
        
        total_files = sum(len(files) for files in folder_files.values())
        
        logger.info(
            f"\nStarting conversion of {total_files} XML files "
            f"in {len(folder_files)} folders"
        )
        
        with track_time("Total conversion process"):
            start_time = time.time()
            results = process_files_batch(session, folder_files)
            total_time = time.time() - start_time
        
        save_conversion_summary(session, results)
        print_final_report(results)
        
        logger.info(f"\nTotal execution time: {total_time:.2f} seconds")
        logger.info("Conversion process completed!")
        
        logger.info("\nTo retrieve DBT project files:")
        logger.info("-- Get all projects:")
        logger.info(f"SELECT DISTINCT project_name FROM {config.OUTPUT_TABLE};")
        logger.info("\n-- Get files for specific project:")
        logger.info(f"SELECT * FROM {config.OUTPUT_TABLE}")
        logger.info("WHERE project_name = 'your_project_name' ORDER BY file_path;")
        
    except KeyboardInterrupt:
        logger.warning("\nProcess interrupted by user")
    except Exception as e:
        logger.error(f"Fatal error: {e}")
        logger.debug(traceback.format_exc())
        raise

if __name__ == "__main__":
    main()